In [1]:
# Import display library
from IPython.display import display, clear_output, Javascript, HTML

#
# Toggles notebook input and prompt cells 
#
display(HTML('''
<script>
code_show=true; 
function code_toggle() {
 if (code_show) {
 $('div.input').hide();
 $('div.prompt').hide();
 } else {
 $('div.input').show();
 $('div.prompt').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
'''))

In [2]:
#
# Define header styles
#
display(HTML('''
<style>
h1.cstm { 
    display: block;
    font-size: 2.5em;
    margin-top: 0;
    margin-bottom: 0;
    margin-left: 0;
    margin-right: 0;
    font-weight: bold;
}
h2.cstm { 
    display: block;
    font-size: 1.5em;
    margin-top: 0;
    margin-bottom: 0;
    margin-left: 0.25in;
    margin-right: 0;
    font-weight: bold;
}
h3.cstm { 
    display: block;
    font-size: 1.25em;
    margin-top: 0;
    margin-bottom: 0;
    margin-left: 0.50in;
    margin-right: 0;
    font-weight: bold;
}
h4.cstm { 
    display: block;
    font-size: 1em;
    margin-top: 0;
    margin-bottom: 0;
    margin-left: 0.75in;
    margin-right: 0;
    font-weight: bold;
}
p.h1cstm {
    margin-left: 0.25in;
    margin-top: 0;
    margin-bottom: 0;
    margin-right: 0;
    font-weight: plain;
    font-size: 1em;
}
p.h2cstm {
    margin-left: 0.50in;
    margin-top: 0;
    margin-bottom: 0;
    margin-right: 0;
    font-weight: plain;
    font-size: 1em;
}
p.h3cstm {
    margin-left: 0.75in;
    margin-top: 0;
    margin-bottom: 0;
    margin-right: 0;
    font-weight: plain;
    font-size: 1em;
}
p.h4cstm {
    margin-left: 1.00in;
    margin-top: 0;
    margin-bottom: 0;
    margin-right: 0;
    font-weight: plain;
    font-size: 1em;
}
</style>
'''
))

<h1 class="cstm">DSM Design Notebook</h1>

In [14]:
# Import system module
import sys

# Import os module
from os import getcwd

# Import types
import types

# Import numerical function module
import numpy as np

# Import scientific:signal function module
from scipy.signal import lfilter

# Import math plotting:pyplot module
import matplotlib.pyplot as plt

# Enable inline interactive plotting
%matplotlib inline

# Import symbolic math module
import sympy as sym
from sympy.printing.latex import LatexPrinter

# Import Widget module
from ipywidgets.widgets import VBox, HBox

# Print using latex mathjax from sympy
sym.init_printing(use_latex='mathjax')

# Append custom python package path to system path
sys.path.append(getcwd)

# Import custom library functions
from customnb.custompsd import psd_welch
from customnb.listtable import ListTable
from customnb.customplots import zplane, autolegcpt, StaticFigureWrapper
from customnb.inputtable import InputFloatTable, InputComplexTable

# Define default notebook figure size
fsize=(5, 3)
fdpi=300

# Function for handling basic polynomial error with nroots
def cnroots(eqn) : 
    try :
        roots=sym.nroots(eqn,n=3,maxsteps=100)
    except sym.polys.polyerrors.PolynomialError :
        return []
    else :
        return roots

<h2 class="cstm">DSM Equation Derivation</h2><hr>
<p class="h2cstm">Derives the delta-sigma modulator signal and noise transfer functions.
</p>

In [4]:
# First, define the symbols utilized for the loop equations.
z=sym.symbols('z')

# Carefully denote that a0-2 must be real
alst=sym.symbols('a:3', real=True)

# Define the forward path transfer function, g(z).
ggg=(z**-1)*(alst[0]/(1-z**-1)+(alst[1]*z**-1)/((1-z**-1)**2)+(alst[2]*z**-2)/((1-z**-1)**3))
gg=sym.simplify(ggg)
g=sym.lambdify(z,gg)

# Define the feedback path transfer function, h(z).
h1=1
h=sym.lambdify(z,h1)

# Define the closed loop transfer function, cltf(z).
cll=(z**-1)*(1+g(z))/(1+g(z)*h(z))
cl=sym.simplify(cll)
cltf=sym.lambdify(z,cl)

# Define the open loop transfer function, oltf(z).
oll=g(z)*h(z)
ol=sym.simplify(oll)
oltf=sym.lambdify(z,ol)

# Define the noise loop transfer function, ntf(z).
nll=1/(1+g(z)*h(z))
nl=sym.simplify(nll)
ntf=sym.lambdify(z,nl)

In [5]:
# Define latex options
largs={'long_frac_ratio':10,
       'fold_frac_powers':True
      }

# Generate equation table
tb1=[["Symbol","Description","Value"],
     ["g(z)","Forward path transfer function.",'$%s$' % sym.latex(g(z),**largs)],
     ["h(z)","Feedback path transfer function.",'$%s$' % sym.latex(h(z),**largs)],
     ["cltf(z)","Closed loop transfer function.",'$%s$' % sym.latex(cltf(z),**largs)],
     ["oltf(z)","Open loop transfer function.",'$%s$' % sym.latex(oltf(z),**largs)],
     ["ntf(z)","Noise transfer function.",'$%s$' % sym.latex(ntf(z),**largs)]
    ]
tcaption="Table I DSM transfer functions."
table=ListTable(tcaption=tcaption,tinput=tb1)
table.show()

In [6]:
# Function counts solutions in listed coeffs with symbols from vlist
def nvarsoln(coeffs=sym.Matrix([]),vlist=()):
    total=0
    for x in coeffs :
        test=False
        for val in vlist : test=test or x.coeff(val)!=0
        if test : total+=1
    return total

def dsmsolve(wd,wdd) :
    # Id suba as a global variable
    global suba
    
    # Assemble target poles/zeros list from input table
    eqsubs=[]
    for key in wd.cvalue :
        eqsubs+=[(key,wd.cvalue[key])]
        
    # Generate a poles model equation
    eqnp=sym.expand(1)
    for sbl in plist : eqnp*=(z-sbl)
    
    # Generate a zeros model equation
    eqnz=sym.expand(1)
    for sbl in zlist : eqnz*=(z-sbl)
    
    # Expand model equations
    eqnpe=sym.expand(sym.N(eqnp.subs(eqsubs)))
    eqnze=sym.expand(sym.N(eqnz.subs(eqsubs)))
    
    # Extract pole and zero model coefficients
    eqnpcoeffs=sym.Matrix(sym.Poly(eqnpe,z).all_coeffs())
    eqnzcoeffs=sym.Matrix(sym.Poly(eqnze,z).all_coeffs())
    eqncoeffs=eqnpcoeffs.row_insert(len(eqnpcoeffs),eqnzcoeffs)
    
    # Solve for the model symbol variables
    allcoeffs=dencoeffs.row_insert(len(dencoeffs),numcoeffs)
    soln=sym.solve(dencoeffs-eqnpcoeffs,alst)
    
    # If no solution found, mark all as None
    if soln==[] : suba={xn:None for xn in alst}  
    else : suba=soln
    
    # Generate table data
    tb1=[["Variable","Value"]]
    for key in suba : 
        if isinstance(suba[key],sym.Float) : tb1+=[[str(key),"%8.6f" % suba[key]]]
        else : tb1+=[[str(key),"%s" % suba[key]]]
    
    # Write to the output table
    wdd.set_input(tb1)
    

# Extract the denominator from the noise transfer function
suba=[]
num, den=nl.as_numer_denom()
dencoeffs=sym.Matrix(sym.Poly(sym.expand(den),z).all_coeffs())
numcoeffs=sym.Matrix(sym.Poly(sym.expand(num),z).all_coeffs())

# Determine number of variable poles and zeroes
npoles=nvarsoln(dencoeffs,alst)
nzeros=nvarsoln(numcoeffs,alst)

# Generates list of possible variable poles and zeros
plist=sym.symbols('px:%d' % npoles)
zlist=sym.symbols('zx:%d' % nzeros)

# Generate output table data
tb1=[["Variable","Value"]]
for key in alst : tb1+=[[str(key),"None" ]]
tcaption="Table III. Calculated filter coefficients."
wd3=ListTable(tcaption=tcaption,tinput=tb1)

# Generate complex input table data
valdef=[0.5+0.5j, 0.5-0.5j, 0.5, 0.5+0.5j, 0.5-0.5j, 0.5]
tb1=[["Variable","Value"]]
for k in range(len(plist)) : 
    tb1+=[[str(plist[k]),"%s" % str(valdef[k])]]
for k in range(len(zlist)) : 
    tb1+=[[str(zlist[k]),"%s" % str(valdef[k])]]
    
# Instantiate input table with wrapper
tcaption="Table II. Target pole and zero locations."
wd2=InputComplexTable(tinput=tb1,tcaption=tcaption)
wd2.setcallback(lambda slf : dsmsolve(slf,wd3))

# Run the solver with initial values
dsmsolve(wd2,wd3)

# Display table widgets side-by-side
bx1=HBox([wd2.getinstance()],padding='20px')
bx2=HBox([wd3.getinstance()],padding='20px')
display(HBox([bx1,bx2]),align='center')

In [7]:
# Input table entries
tcaption="Table IV. Designed digital filter coefficients."
tinput =[["Variable","Description","Value","Units"],
         ["fs","Sample frequency.","20e6","Hz"]
        ]

# Instantiate input table with wrapper
wwd2=InputFloatTable(tinput=tinput,tcaption=tcaption)
wwd2.show()

In [8]:
# Set sample frequency
fsam=wwd2.value['fs']

# Generate log space
freq=np.logspace(3,np.log10(fsam/2),num=1000)
zin=np.exp(2j*np.pi*freq/fsam)

# Generate output functions with variable substitutions
cltfo=sym.lambdify(z,sym.Abs(cltf(z).subs(suba)),"numpy")
ntfo=sym.lambdify(z,sym.Abs(ntf(z).subs(suba)),"numpy")

#
# Generate step plot with outval and outf
#
w1=StaticFigureWrapper(flabel="Figure1",fsize=fsize,fdpi=fdpi)
w1.ax.semilogx(freq,20*np.log10(cltfo(zin)),label='CLTF',linewidth=2)
w1.ax.semilogx(freq,20*np.log10(ntfo(zin)),label='NTF',linewidth=2)
w1.ax.set_xlim([freq[0],fsam/2])
w1.ax.set_ylim([-200,10])
w1.ax.grid(True,which="both",ls="-")
w1.ax.set_xlabel('frequency (Hz)')
w1.ax.set_ylabel(' Output Magnitude (dB)')

# Autogenerate legend and figure caption
autolegcpt(w1.fig,"Fig. 1 Calculated DSM closed loop and noise transfer functions.")

#
# Figure 2: Open loop gain and phase
#
# Generate output functions with variable substitutions
oltfo=sym.lambdify(z,oltf(z).subs(suba),"numpy")

#
# Generate step plot with outval and outf
#
w2=StaticFigureWrapper(flabel="Figure2",fsize=fsize,fdpi=fdpi)
w2.ax.semilogx(freq,
               20*np.log10(np.abs(oltfo(zin))),
               label='Calculated OLTF gain',
               color='blue',
               linewidth=2)
w2.ax.set_xlim([freq[0],fsam/2])
w2.ax.get_xaxis().get_major_formatter().labelOnlyBase=False 
w2.ax.set_ylim([-20,200])
w2.ax.grid(True,which="both",ls="-")
w2.ax.set_xlabel('frequency (Hz)')
w2.ax.set_ylabel('Gain (dB)')

# Plot phase on secondary y-axis
ax3=w2.ax.twinx()
ax3.semilogx(freq,
             np.angle(oltfo(zin),deg=True),
             label='Calculated OLTF phase',
             color='red',
             linewidth=2
            )
ax3.set_ylim([-180,180])
ax3.yaxis.set_ticks(np.arange(-180, 181, 45))
ax3.set_ylabel('Phase (degrees)')

# Autogenerate legend and figure caption
autolegcpt(w2.fig,"Fig. 2 Calculated DSM open loop gain and phase.")

#
# Figure 3: CLTF Pole Zero Plot
#
w3=StaticFigureWrapper(flabel="Figure3",fsize=fsize,fdpi=fdpi)

numer,denom=cltf(z).subs(suba).as_numer_denom()
xzeros=cnroots(numer)
xpoles=cnroots(denom)
zplane(ax=w3.ax,z=xzeros,p=xpoles,
       ctitle="Fig. 3 Calculated DSM CLTF pole-zero plot.",
       fsize=10
      )

#
# Figure 4: NTF Pole Zero Plot
#
w4=StaticFigureWrapper(flabel="Figure4",fsize=fsize,fdpi=fdpi)

numer,denom=ntf(z).subs(suba).as_numer_denom()
xzeros=cnroots(numer)
xpoles=cnroots(denom)
zplane(ax=w4.ax,z=xzeros,p=xpoles,
       ctitle="Fig. 4 Calculated DSM NTF pole-zero plot.",
       fsize=10
      )

# Display plot widgets
display(VBox([HBox([w1.render(),w2.render()],align="start"),
              HBox([w3.render(),w4.render()],align="start")
             ]))

<h2 class="cstm">Measurements</h2><hr>
<p class="h2cstm">Reads output file and makes measurements from data.
</p>

In [9]:
# Input table entries
tcaption="Table IV. Output measurement parameters."
tinput =[["Variable","Description","Value","Units"],
         ["fs","Sample frequency.","20e6","Hz"],
         ["dlen","Data length.","65536","#"],
         ["mlen","Measure length.","65536","#"],
         ["pseg","Pwelch method segments.","16","#"]
        ]

# Instantiate input table with wrapper
wwd3=InputFloatTable(tinput=tinput,tcaption=tcaption)
wwd3.show()

In [10]:
#
# Constants
#
mlen=int(wwd3.value['mlen']) # Set measurement length
dlen=int(wwd3.value['dlen']) # Set display length
fs=wwd3.value['fs']          # Sample frequency
pseg=int(wwd3.value['pseg']) # pwelch segments

#
# Reads simulation output file 
#
outval,rval=np.loadtxt('test.txt', delimiter='\t',unpack=True)

# Filter output with simple averaging
outf=lfilter(np.ones(100)/100,1,outval)

# Measurements of outval
maxval=np.max(outval)
minval=np.min(outval)
mlen=np.min((len(outval),mlen))
meanval=np.mean(outval[len(outval)-mlen:len(outval)])

# Capture PSDs, frequencies
fp1,fp2,Pxx1,Pxx2=psd_welch(outval,fs,pseg)
    
# Compute quantization and dither noise PSD
inns=1/(12*fs)+1/(12*fs)

# Convert PSD to dB scale, scale for quantization and dither
IPxx1=10*np.log10(Pxx1)-10*np.log10(inns)
IPxx2=10*np.log10(Pxx2)-10*np.log10(inns)

# Finds the max PSD 
maxpsd=np.max(lfilter(np.ones(100)/100,1,IPxx2))

<h3 class="cstm">Output Tables</h3>

In [11]:
# Table array
ltable=[['Symbol','Description','Value','Units'],
        [r'${{out}_{max}}$','Maximum output value',"%2d" % maxval,'#'],
        [r'${{out}_{min}}$','Minimum output value',"%2d" % minval,'#'],
        [r'${{out}_{mean}}$','Mean output value (%d samples)' % mlen,"%4.6f" % meanval,'#'],
        [r'${{peak}_{max}}$','Maximum PSD peaking',"%4.3f" % maxpsd,'dB']
       ]

# Generate HTML table
table=ListTable(tcaption="Table V. DSM output measurements.",tinput=ltable)
table.show()

<h3 class="cstm">Plots</h3>

In [12]:
# Generate time axis from output length
olen=len(outval)
x = np.arange(1,olen+1)

# Generate figure for plots
w1=StaticFigureWrapper(flabel="Figure5",fsize=fsize,fdpi=fdpi)

#
# Generate step plot with outval and outf
#
w1.ax.step(x[olen-dlen:olen],outval[olen-dlen:olen],label='DSM output',linewidth=1)
w1.ax.step(x[olen-dlen:olen],outf[olen-dlen:olen],label='Averaged DSM output',linewidth=1)
w1.ax.set_xlim([olen-dlen,olen])
w1.ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
w1.ax.set_ylim([-4,3])
w1.ax.set_xlabel('time (#)')
w1.ax.set_ylabel('Output value (#)')

# Autogenerate legend and figure caption
autolegcpt(w1.fig,"Fig. 5 DSM actual and average output vs. time step.")

#
# Plot noise spectrums
#
w2=StaticFigureWrapper(flabel="Figure6",fsize=fsize,fdpi=fdpi)

# Generate axes object and plot
w2.ax.semilogx(fp1, IPxx1,label='DSM output (pwelch)')
w2.ax.semilogx(fp2, IPxx2,label='DSM output (pwelch %2d seg)' % pseg)
w2.ax.semilogx(freq,20*np.log10(ntfo(zin)),label='Calculated NTF',linewidth=1)
w2.ax.grid(True,which="both",ls="-")
w2.ax.set_xlim([1e3, fs/2])
w2.ax.set_ylim([-200,20])
w2.ax.set_xlabel('frequency (Hz)')
w2.ax.set_ylabel(r'PSD $\mathregular{(\#^2/Hz)}$')

# Autogenerate legend and figure caption
autolegcpt(w2.fig,"Fig. 6 DSM output PSD.")

# Display plot widgets
display(HBox([w1.render(),w2.render()],align="start"))

<h2 class="cstm"> Python/Jupyter Compatibilty </h2><hr>
<p class="h2cstm">Lists all modules obtained from globals().  Doesn't necessarily capture all modules in use.</p>

In [15]:
#
# Checks the global import modules and versions
#
# Find list of module names in global
mlist=[]
for name, module in globals().items():
    if isinstance(module, types.ModuleType):
        mlist+=[module.__name__.split('.')[0]]
mlist=list(set(mlist))

# Check unique module versions in mlist in sys.modules
moduleinfo=[['Module','Version']]
for mnm in mlist:
    mod=sys.modules[mnm]
    if hasattr(mod, '__version__'): 
        moduleinfo+= [[mod.__name__, mod.__version__]]
    elif hasattr(mod, '__VERSION__'):
        moduleinfo+= [[mod.__name__, mod.__VERSION__]]
    elif hasattr(module, 'VERSION'):
        moduleinfo+= [[mod.__name__, mod.VERSION]]
    elif hasattr(mod, 'version_info'):
        if (mod.__name__=='sys') :
            moduleinfo+= [['Python', "%d.%d.%d" % mod.version_info[:3]]]
        else :
            moduleinfo+= [[mod.__name__, "%d.%d.%d" % mod.version_info[:3]]]
    elif hasattr(mod, 'version'):
        moduleinfo+= [[mod.__name__, mod.version]]
    else :
        if (mod.__name__!='__builtin__'):
            moduleinfo+= [[mod.__name__, '']]
    
# Generate HTML table
table=ListTable(tcaption="Table VI. Imported python/jupyter modules.",tinput=moduleinfo)
table.show()